<a href="https://colab.research.google.com/github/Poorna-A/add/blob/master/Data_Vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets
!pip install pyxlsb
!pip install jupyter-dash
!pip install dash
!pip install plotly
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
import matplotlib.pyplot as plt
from dash import Dash, dcc, html, Input, Output

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
location = r'/content/drive/MyDrive/Poorna  Akhil Workspace/Skill Networking/June 17 Alloc.xlsb'
df = pd.read_excel(location,sheet_name = 'Sheet1')

In [4]:
df.columns

Index(['Employee ID', 'Employee Name', 'Employee Type', 'OBU',
       'OBU Description', 'Customer ID', 'Customer Name', 'Project ID',
       'Project Description', 'Program Description', 'Program Manager',
       'Contract Category', 'Project Manager ID', 'Project Manager Name',
       'Project Start Date', 'Project End Date', 'Start Date', 'End Date',
       'Country', 'Location', 'Location Descr', 'JobCode', 'JobCode Descr',
       'Billable?', 'Hire Reason', 'Client Project Role',
       'Client Project Role Desc', 'Alias Name', 'Amount', 'Rate Per Role',
       'Frequency', 'Currency', 'Allocation %tage', 'Submitted By',
       'Submission Date', 'Reason for Allocation', 'Allocation Status',
       'OptimUST', 'Manager ID', 'Manager Name', 'Hire Date',
       'Termination Date', 'Employee Status', 'Business Unit', 'BU Descr',
       'Department ID', 'Department Descr', 'Comments',
       'Allocation Modified Date'],
      dtype='object')

In [7]:
# Count employees per country
country_count = df['Country'].value_counts().reset_index()
country_count.columns = ['Country', 'Employee Count']

Error in total allocation data... Need to check it...

In [8]:
# Calculate total allocation percentage for each employee
df['Allocation %tage'] = df['Allocation %tage'].astype(float)
total_allocation = df.groupby('Employee ID')['Allocation %tage'].sum().reset_index()
total_allocation.columns = ['Employee ID', 'Total Allocation']

# Filter employees with total allocation less than 100%
under_allocated = total_allocation[total_allocation['Total Allocation'] < 100]

# Merge with the original dataframe to get country and location info
under_allocated = pd.merge(under_allocated, df[['Employee ID', 'Country', 'Location', 'Allocation %tage']], on='Employee ID')


In [9]:
# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='country-graph'),
    dcc.Graph(id='location-graph'),
    dcc.Graph(id='under-allocation-graph'),
    dcc.Store(id='selected-country', storage_type='memory'),
    dcc.Store(id='selected-location', storage_type='memory')
])

@app.callback(
    Output('country-graph', 'figure'),
    Input('country-graph', 'clickData')
)
def update_country_graph(click_data):
    fig = px.pie(country_count, names='Country', values='Employee Count', title='Employees per Country')
    return fig

Location Graph is not responding.. .. And So that the allocation graph is also blank

In [10]:
@app.callback(
    Output('location-graph', 'figure'),
    [Input('country-graph', 'clickData'),
     Input('selected-country', 'data')]
)
def update_location_graph(click_data, selected_country):
    ctx = dash.callback_context
    if ctx.triggered and ctx.triggered[0]['prop_id'].split('.')[0] == 'country-graph':
        selected_country = click_data['points'][0]['label']
    if selected_country:
        filtered_df = df[df['Country'] == selected_country]
        location_count = filtered_df['Location'].value_counts().reset_index()
        location_count.columns = ['Location', 'Employee Count']
        fig = px.bar(location_count, x='Location', y='Employee Count', title=f'Employees in {selected_country}')
        return fig
    return px.bar(title='Select a country to see the distribution of employees by location.')


In [11]:
@app.callback(
    Output('under-allocation-graph', 'figure'),
    [Input('location-graph', 'clickData'),
     Input('selected-country', 'data')]
)
def update_under_allocation_graph(click_data, selected_country):
    ctx = dash.callback_context
    if ctx.triggered and ctx.triggered[0]['prop_id'].split('.')[0] == 'location-graph':
        selected_location = click_data['points'][0]['label']
        filtered_under_allocated = under_allocated[(under_allocated['Country'] == selected_country) &
                                                   (under_allocated['Location'] == selected_location)]
        fig = px.bar(filtered_under_allocated, x='Employee ID', y='Total Allocation',
                     title=f'Employees with Less Than 100% Allocation in {selected_location}')
        return fig
    return px.bar(title='Select a location to see employees with less than 100% allocation.')


In [12]:
@app.callback(
    Output('selected-country', 'data'),
    Input('country-graph', 'clickData')
)
def store_selected_country(click_data):
    if click_data:
        selected_country = click_data['points'][0]['label']
        return selected_country
    return None

@app.callback(
    Output('selected-location', 'data'),
    Input('location-graph', 'clickData')
)
def store_selected_location(click_data):
    if click_data:
        selected_location = click_data['points'][0]['label']
        return selected_location
    return None


In [13]:
# Run the app
app.run_server(mode='inline')

<IPython.core.display.Javascript object>